In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import random
import scipy

from PIL import Image
from os import getcwd

from tqdm import tqdm
from mpl_toolkits.axes_grid1 import make_axes_locatable

%matplotlib inline

input_dir = getcwd() + '/img/input_samples/'

In [ ]:
###########################################################################################################################
#                                                      Array utils                                                        #
###########################################################################################################################


def normalize(arr, zero_padding=False, offset_coef=0.001):
    min_val = np.min(arr)
    max_val = np.max(arr)
    offset = offset_coef * abs(min_val)
    new_arr = arr - min_val if zero_padding else arr - min_val + offset
    d = (max_val - min_val) if zero_padding else (max_val - min_val + offset)
    new_arr = new_arr / d
    return new_arr


def normalize_img(arr):
    arr_min = np.min(arr)
    arr_max = np.max(arr)
    if arr_min > 0 and arr_max < 255: 
        return arr
    else:
        arr_norm = 255 * (arr - arr_min) / (arr_max - arr_min)
        return arr_norm


def threshold_arr_1d(arr, th_val, use_abs=False):
    th_val = abs(th_val) if use_abs else th_val
    for i in range(len(arr)):
        if arr[i] < th_val:
            arr[i] = th_val
    return arr
    
    
def threshold_arr_2d(arr, th_val, use_abs=False):
    for i in range(arr.shape[0]):
        arr[i, :] = threshold_arr_1d(arr[i, :], th_val, use_abs)
    return arr


###########################################################################################################################
#                                                       FFT utils                                                         #
###########################################################################################################################


def find_ft_1d(img):
    ft = np.fft.fft(img)
    return np.fft.fftshift(ft)


def find_ift_1d(ft):
    ift = np.fft.ifftshift(ft)
    return np.fft.ifft(ift)


def find_ft_2d(img):
    ft = np.fft.fft2(img)
    return np.fft.fftshift(ft)


def find_ift_2d(ft):
    ift = np.fft.ifftshift(ft)
    return np.fft.ifft2(ift)


def freq_numbers_1d(size):
    if size % 2:
        return np.arange(-(size // 2), size // 2 + 1, 1) 
    else:
        return np.arange(-(size // 2), size // 2, 1) 
    

def freq_arr_1d(size, step=1):
    freq = freq_numbers_1d(size) 
    return freq / step / size


def freq_numbers_2d(size):
    x_size, y_size = size
    x_freq_numbers = freq_numbers_1d(x_size)
    y_freq_numbers = freq_numbers_1d(y_size)
    return x_freq_numbers, y_freq_numbers
    

def freq_arr_2d(size, x_step=1, y_step=1):
    x_size, y_size = size
    x_freq_numbers, y_freq_numbers = freq_numbers_2d(size) 
    x_freq = x_freq_numbers / x_step / x_size
    y_freq = y_freq_numbers / y_step / y_size
    return x_freq, y_freq


###########################################################################################################################
#                                                   Frequency domain filters                                              #
###########################################################################################################################


def freq_filter_2d(x_freq, y_freq):
    x, y = np.meshgrid(x_freq, y_freq)
    f = np.hypot(x, y)
    return f


# should be deprecated or changed
"""
def freq_filter(x_freq, y_freq, factor=2.4):
    eps = 10 ** -8
    x, y = np.meshgrid(x_freq, y_freq)
    f = np.hypot(x, y)
    f = f ** factor + eps
    return normalize(1 / f)
"""


def freq_pink_filter_2d(x_freq, y_freq, factor=1, zero_padding=False, no_mean=False):
    x, y = np.meshgrid(x_freq, y_freq)
    f = np.hypot(x, y)
    f = 1 / (1 + np.abs(f))
    f = f ** factor
    f = np.where(f==1, 0, f) if no_mean else f
    return normalize(f, zero_padding)


def freq_pink_filter_1d(x_freq, factor=1, no_mean=False):
    f = 1 / (1 + np.abs(x_freq))
    f = f ** factor
    f = np.where(f==1, 0, f) if no_mean else f
    return normalize(f, zero_padding)


# replaced by no_mean flag in freq_filter_1d
"""
def freq_filter_1d_a(x_freq, factor=1):
    f = 1 / np.where(x_freq == 0, 1, np.abs(x_freq))
    f = f ** factor
    f[len(x_freq) // 2] = 0
    return f
"""


def freq_sharp_round_filter_2d(x_freq, y_freq, radius, low_pass_filter=True):
    f = np.ones((np.size(x_freq), np.size(y_freq)))
    check = gt if low_pass_filter else lt
    rr = radius ** 2

    for i, x in enumerate(x_freq):
        for j, y in enumerate(y_freq):
            if check(x ** 2 + y ** 2, rr):
                f[i, j] = 0
    return f


def freq_sharp_square_filter(x_freq, y_freq, width, angle=0, low_pass_filter=True):
    f = np.ones((np.size(x_freq), np.size(y_freq)))
    check = np.greater if low_pass_filter else np.less
    angle_radians = math.radians(angle)
    rotation_matrix = np.array([[math.cos(angle_radians), -math.sin(angle_radians)],
                                [math.sin(angle_radians), math.cos(angle_radians)]])
    
    for i, x in enumerate(x_freq):
        for j, y in enumerate(y_freq):
            rotated_x, rotated_y = np.dot(rotation_matrix, np.array([x, y])) 

            if check(abs(rotated_x) + abs(rotated_y), width):
                f[i, j] = 0
    return f


###########################################################################################################################
#                                                Spatial domain utils (NOT REWORKED)                                      #
###########################################################################################################################


def spatial_smooth_filter(x_size, y_size, depth, horiz=True):
    values = np.linspace(0, 1, depth)
    values = 6 * values ** 5 - 15 * values ** 4 + 10 * values ** 3
    values = 1 - values
    if horiz:
        kernel = np.tile(values, (y_size, 1))
    else:
        kernel = values[:, np.newaxis] * np.ones((1, x_size))   
    return kernel


def make_img_transition_x(img, depth, is_dx_pos=True):
    y_size, x_size = img.shape
    additional_img = gen_cloud(x_size + depth, y_size)   
    transition_kernel = spatial_smooth_filter(x_size, y_size, depth)     
    
    new_img = np.copy(img)
    if is_dx_pos:
        new_img[:, -depth:x_size] = img[:, -depth:x_size] * transition_kernel + \
                                additional_img[:, 0:depth] * (1 - transition_kernel)
        return new_img, additional_img[:, depth:]    
    else:
        transition_kernel = np.fliplr(transition_kernel)
        new_img[:, 0:depth] = img[:, 0:depth] * transition_kernel + \
                          additional_img[:, -depth:] * (1 - transition_kernel)  
        return new_img, additional_img[:, 0:-depth]    


def make_img_transition_y(img, depth, is_dy_pos=True):
    y_size, x_size = img.shape
    additional_img = gen_cloud(x_size, y_size + depth)   
    transition_kernel = spatial_smooth_filter(x_size, y_size, depth, horiz=False)
        
    new_img = np.copy(img)
    if is_dy_pos:
        new_img[-depth:x_size, :] = img[-depth:x_size, :] * transition_kernel + \
                                additional_img[0:depth, :] * (1 - transition_kernel)
        return new_img, additional_img[depth:, :]    
    else:
        transition_kernel = np.flipud(transition_kernel)
        new_img[0:depth, :] = img[0:depth, :] * transition_kernel + \
                          additional_img[-depth:, :] * (1 - transition_kernel)  
        return new_img, additional_img[0:-depth:1, :]    



###########################################################################################################################
#                                                       Other (NOT REWORKED)                                              #
###########################################################################################################################


def normalize_psd(original_magn, other_magn):
    original_psd = original_magn ** 2
    other_psd = other_magn ** 2
    return (np.sum(original_psd) / np.sum(other_psd)) ** 0.5


def fit_clement(x_freq, y_freq, alpha1, alpha2, eta=0.1, angle=3):
    x, y = np.meshgrid(x_freq, y_freq)
    f = np.hypot(x, y )
    fp = abs(angle * x  + y)
    f = np.sqrt((1 - eta) * f ** 2 + eta * fp ** 2)    
    f = (1 + abs(f)) ** (-alpha1) + 0.02 * (1 + abs(f)) ** (-alpha2)
    return f


def surrogates(x, ns, tol_pc=5., verbose=True, maxiter=1E6, sorttype="quicksort"):
    # as per the steps given in Lancaster et al., Phys. Rep (2018)
    nx = x.shape[0]
    xs = np.zeros((ns, nx))
    maxiter = 10000
    ii = np.arange(nx)

    # get the fft of the original array
    x_amp = np.abs(np.fft.fft(x))
    x_srt = np.sort(x)
    r_orig = np.argsort(x)

    # loop over surrogate number
    pb_fmt = "{desc:<5.5}{percentage:3.0f}%|{bar:30}{r_bar}"
    pb_desc = "Estimating IAAFT surrogates ..."
    for k in tqdm(range(ns), bar_format=pb_fmt, desc=pb_desc,
                  disable=not verbose):

        # 1) Generate random shuffle of the data
        count = 0
        r_prev = np.random.permutation(ii)
        r_curr = r_orig
        z_n = x[r_prev]
        percent_unequal = 100.

        # core iterative loop
        while (percent_unequal > tol_pc) and (count < maxiter):
            r_prev = r_curr

            # 2) FFT current iteration yk, and then invert it but while
            # replacing the amplitudes with the original amplitudes but
            # keeping the angles from the FFT-ed version of the random
            y_prev = z_n
            fft_prev = np.fft.fft(y_prev)
            phi_prev = np.angle(fft_prev)
            e_i_phi = np.exp(phi_prev * 1j)
            z_n = np.fft.ifft(x_amp * e_i_phi)

            # 3) rescale zk to the original distribution of x
            r_curr = np.argsort(z_n, kind=sorttype)
            z_n[r_curr] = x_srt.copy()
            percent_unequal = ((r_curr != r_prev).sum() * 100.) / nx

            # 4) repeat until number of unequal entries between r_curr and 
            # r_prev is less than tol_pc percent
            count += 1

        if count >= (maxiter - 1):
            print("maximum number of iterations reached!")

        xs[k] = np.real(z_n)

    return xs


def adjust_freq_1d(img):
    return np.append(img, abs(img[0]))


def adjust_img_1d(img):
    return np.append(img, img[0])
        
        
def gen_cloud(x_size, y_size, factor=2.4):
    xx = np.linspace(-x_size / 2, x_size / 2, x_size)
    yy = np.linspace(-y_size / 2, y_size / 2, y_size)
    whitenoise = np.random.normal(0, 1, (y_size, x_size))
    cloud_freq = find_ft_2d(whitenoise)  
    kernel = freq_filter(xx, yy, factor=factor)
    cloud_freq_filtered = cloud_freq * kernel
    cloud_spatial = find_ift_2d(cloud_freq_filtered).real
    return normalize_img(cloud_spatial)


# Reworked
def show_images(*images, vrange=None, x_fig_size=10, y_fig_size=10, cmap='gray', graphs_per_row=2):
    row_num = len(images) // graphs_per_row + 1 if len(images) % graphs_per_row else len(images) // graphs_per_row
    col_num = len(images) // row_num + 1 if len(images) % row_num else len(images) // row_num
    images_len = len(images)
    
    f, axes = plt.subplots(row_num, col_num, figsize=(x_fig_size, y_fig_size))  
    if row_num == 1 and col_num == 1:
        axes = np.array([axes])
    
    for index, ax in enumerate(axes.flatten()):
        if index < images_len:
            if vrange:
                vmin, vmax = vrange
                im = ax.imshow(images[index], cmap=cmap, vmin=vmin, vmax=vmax, aspect='equal')
                
            else:
                im = ax.imshow(images[index], cmap=cmap, aspect='equal')
            divider = make_axes_locatable(ax)
            cax = divider.append_axes("right", size="5%", pad=0.05)
            plt.colorbar(im, cax=cax)
        else:
            ax.axis('off')

    plt.tight_layout()
    return f, axes

    
def show_surfaces(*surfaces, axes=None, cmap=None, colorscale='Thermal', showscale=True):  
    if cmap:
        cmin, cmax = cmap
    else:
        if axes:
            z_data = [surf[-1] for surf in surfaces]
        else:
            z_data = surfaces
        cmin, cmax = np.min(z_data), np.max(z_data)
    
    fig = go.Figure()
    for surf in surfaces:
        if axes: 
            x, y = axes
            z = surf
        else:
            x, y, z = surf
        fig.add_trace(go.Surface(x=x, y=y, z=z, cmin=cmin, cmax=cmax, colorscale=colorscale, showscale=showscale))
        showscale = False
    return fig
    

def lin_regression(x, y):
    # y - original img
    # x - restored img
    num = np.mean(x * y) - np.mean(x) * np.mean(y)
    denum = np.mean(x ** 2) - np.mean(x) ** 2
    a = num / denum
    b = np.mean(y) - a * np.mean(x)
    return a, b


def lin_phase(start, end, size):
    pos_freq = np.linspace(start, end, size // 2)
    neg_freq = -pos_freq[::-1]
    return np.append(neg_freq, pos_freq)

### Cumulus test

In [ ]:
img = Image.open(input_dir + '2.jpg').convert('L')
img -= np.mean(img)
img_fr = find_ft_2d(img)

x_size, y_size = img.shape
x_shift = x_size // 2
y_shift = y_size // 2
xx = freq_numbers_1d(x_size)
yy = freq_numbers_1d(y_size)
x, y = np.meshgrid(xx, yy)
x_ax_size = 10
y_ax_size = 8.5

magn_raw = np.abs(img_fr)
magn = normalize(magn_raw)
magn_factor = normalize_psd(magn_raw, magn)
phase = np.angle(img_fr)
restored_img = find_ift_2d(magn_factor * magn * np.exp(1j * phase)).real

f, ax = show_images(img, np.log10(magn), phase, restored_img, x_fig_size=x_ax_size, y_fig_size=y_ax_size)

In [ ]:
phase_fr = find_ft_2d(phase)
phase_magn = np.abs(phase_fr)
phase_angle = np.angle(phase_fr)
restored_phase = find_ift_2d(phase_magn * np.exp(1j * phase_angle)).real

f, ax = show_images(np.log10(phase_magn), phase_angle, restored_phase, x_fig_size=x_ax_size, y_fig_size=y_ax_size)

In [ ]:
f_1 = show_surfaces(np.log10(magn), axes=(xx, yy))
# f_1.update_layout(scene = dict(zaxis = dict(range=[-5, 0])))
# f_1.show()

In [ ]:
# f_2 = show_surfaces(phase[x_size // 2 - 10: x_size // 2 + 10, y_size // 2 - 10: y_size // 2 + 10], axes=(list(range(20)), list(range(20))))
f_2 = show_surfaces(phase, axes=(xx, yy))
# f_2.show()

### Generating pseudo random phase trying to mimic same phase distribution along each axis

In [ ]:
new_phase_y = np.zeros((x_size, y_size))
for i in range(y_size):
    new_phase_y[i, :] = surrogates(phase[i, :], 1)
    
new_phase_x = np.zeros((x_size, y_size))
for i in range(x_size):
    new_phase_x[:, i] = surrogates(phase[:, i], 1)

# New phase and 1/f kernel
new_phase_sur = new_phase_x + new_phase_y 

# Restoring image using original phase and generated phase
restored_img = find_ift_2d(magn_factor * magn * np.exp(1j * new_phase_sur)).real

f, ax = show_images(new_phase_sur, restored_img, x_fig_size=x_ax_size, y_fig_size=y_ax_size)

In [ ]:
# Restoring image using 1/f kernel for magnitude and new phase 
# Results are equal to regular FFT syntesis

kernel = freq_pink_filter_2d(xx, yy)
kernel_magn_factor = normalize_psd(magn_raw, kernel)
restored_img = find_ift_2d(kernel_magn_factor * kernel * np.exp(1j * new_phase_sur)).real

f, ax = show_images(np.log10(kernel), restored_img, x_fig_size=x_ax_size, y_fig_size=y_ax_size)

### Restoring image using linearly changing phase

In [ ]:
initial_phase = [np.pi / 3, 2 * np.pi / 3, np.pi, np.pi / 2, 3 * np.pi / 2]
initial_phase += [-i for i in initial_phase]
whitenoise = np.random.normal(0, 1, (y_size, x_size))
whitenoise_fr = find_ft_2d(whitenoise)
lin_random_phase = np.angle(whitenoise_fr)

width = 5
scale = 0.2
alpha_x = 1.4
alpha_y = 1.
step = 1

for i in range(-width, width + 1, step):
    initial_phase_x = random.choice(initial_phase)
    initial_phase_y = random.choice(initial_phase)
    freq_y = lin_phase(initial_phase_y, initial_phase_y + random.choice(initial_phase), y_size)
    freq_x = lin_phase(initial_phase_x, initial_phase_x + random.choice(initial_phase), x_size)
    lin_random_phase[lin_random_phase.shape[0] // 2 + i, :] = freq_y
    lin_random_phase[:, lin_random_phase.shape[1] // 2 + i] = freq_x

restored_img_fr = magn_factor * magn * np.exp(1j * lin_random_phase)
restored_img = find_ift_2d(restored_img_fr).real

f, ax = show_images(lin_random_phase, restored_img, x_fig_size=x_ax_size, y_fig_size=y_ax_size)

In [ ]:
magn_factor = normalize_psd(magn_raw, kernel)
restored_img_fr = magn_factor * kernel * np.exp(1j * lin_random_phase)
restored_img = find_ift_2d(restored_img_fr).real

f, ax = show_images(lin_random_phase, restored_img, x_fig_size=x_ax_size, y_fig_size=y_ax_size)

### 1D regression

In [ ]:
# Custom regression formulas

# y_log = np.log(y_vals)
# x_log = np.log(1 + x_vals)

# alpha = -2 * np.sum(y_log * x_log) / np.sum(x_log ** 2) 
# alpha

In [ ]:
# Approximation along Y-axis

alpha = 1.
dy = 1
factor_y = [1, 0.2, 0.2, 0.15, 0.25, 0.08]
const_y = [0.0019, 0.0008, 0.0015, 0.001, 0.0012, 0.0008]

y_vals = magn[x_size // 2 + dy, magn.shape[1] // 2:] 
x_vals = np.array(range(len(y_vals)))

w_y = factor_y[abs(dy)] * ((1 + abs(x_vals)) ** (-alpha) + const_y[abs(dy)])
# kernel = freq_pink_filter_2d(xx, yy, factor=1)
# kernel[magn.shape[0] // 2, :] = w

plt.plot(x_vals, w_y)
plt.plot(x_vals[1:], y_vals[1:])
plt.yscale('log')

In [ ]:
# Approximation along X-axis

alpha = 1.45
dx = 1
factor_x = [1, 1, 1, 1, 1, 1]
const_x = [0.00015, 0.00015, 0.00015, 0.00015, 0.00015, 0.00015]

y_vals = magn[magn.shape[0] // 2:, y_size // 2 + dx] 
x_vals = np.array(range(len(y_vals)))
w_x = factor_x[abs(dx)] * ((1 + abs(x_vals)) ** (-alpha) + const_x[abs(dx)])

plt.plot(x_vals, w_x)
plt.plot(x_vals[0:], y_vals[0:])
plt.yscale('log')

In [ ]:
# Approximation along Y = X

alpha1 = 1.8
alpha2 = 1.0

l = magn.shape[0] // 2
y_vals = np.zeros(l)

for i in range(l):
    y_vals[i] = magn[l + i, l + i]
x_vals = np.array(range(len(y_vals))) * 2 ** 0.5

y_log = np.log10(y_vals)
x_log = np.log10(1 + x_vals)

w_diag = (1 + abs(x_vals)) ** (-alpha1) + 0.02 * (1 + abs(x_vals)) ** (-alpha2)
# kernel[:, magn.shape[1] // 2] = w

plt.plot(x_vals, w_diag)
plt.plot(x_vals[1:], y_vals[1:])
plt.yscale('log')

In [ ]:
# Combined kernel with previous approximation results

width = 5
scale = 0.2
alpha_x = 1.4
alpha_y = 1.
step = 1

# kernel = freq_pink_filter_2d(xx, yy, factor=0.5)
kernel = fit_clement(xx, yy, alpha1, alpha2)
# kernel = np.zeros((x_size, y_size))

for i in range(-width, width + 1, step):
    w_y = factor_y[abs(i)] * ((1 + abs(yy)) ** (-alpha_y) + const_y[abs(i)])
    w_x = factor_x[abs(i)] * ((1 + abs(xx)) ** (-alpha_x) + const_x[abs(i)])
    kernel[magn.shape[0] // 2 + i, :] = w_y
    kernel[:, magn.shape[1] // 2 + i] = w_x
    
# kernel[magn.shape[0] // 2, :] = w_y 
# kernel[:, magn.shape[1] // 2] = w_x 
kernel = normalize(kernel)

f = show_surfaces(np.log10(kernel), axes=(xx, yy))
# fig.update_layout(scene=dict(zaxis = dict(range=[-5, 0])))
# f.show()

In [ ]:
# Restring image using approximated kernel and original phase
restored_img_1 = find_ift_2d(kernel * np.exp(1j * phase)).real
a, b = lin_regression(restored_img_1, img)
restored_img_1 = a * restored_img_1 + b

# Restring image using approximated kernel and random phase
whitenoise = np.random.normal(0, 1, (y_size, x_size))
random_phase = np.angle(find_ft_2d(whitenoise))
kernel_magn_factor = normalize_psd(magn_raw, kernel)
restored_img_2 = find_ift_2d(kernel_magn_factor * kernel * np.exp(1j * random_phase)).real

# Restring image using approximated kernel and linear phase
restored_img_3 = find_ift_2d(kernel_magn_factor * kernel * np.exp(1j * lin_random_phase)).real

# plt.imshow(restored_img_1, cmap='gray', vmin=np.min(img), vmax=np.max(img))
f, ax = show_images(restored_img_1, restored_img_2, restored_img_3, x_fig_size=x_ax_size, y_fig_size=y_ax_size)

### 2D regression

In [ ]:
y_vals = magn
x_vals = freq_filter_2d(xx, yy)

y_log = np.log10(y_vals)
x_log = np.log10(1 + x_vals)

alpha = -2 * np.sum(y_log * x_log) / np.sum(x_log ** 2) 
kernel = freq_pink_filter_2d(xx, yy, factor=alpha / 2)
print(f"alpha = {alpha / 2}")

f = show_surfaces(np.log10(kernel), np.log10(magn), axes=(xx, yy))
# f.show()

In [ ]:
# Restoring image using approximated kernel and original phase 
kernel_magn_factor = normalize_psd(magn_raw, kernel)
restored_image_1 = find_ift_2d(kernel_magn_factor * kernel * np.exp(1j * phase)).real
a, b = lin_regression(restored_image_1, img)
restored_image_1 = a * restored_image_1 + b

# Restoring image using approximated kernel and random phase 
restored_image_2 = find_ift_2d(kernel_magn_factor * kernel * np.exp(1j * random_phase)).real

# Restoring image using approximated kernel and linear phase 
restored_image_3 = find_ift_2d(kernel_magn_factor * kernel * np.exp(1j * lin_random_phase)).real

f, ax = show_images(restored_image_1, restored_image_2, restored_image_3, vrange=(np.min(img), np.max(img)), 
                    x_fig_size=x_ax_size, y_fig_size=y_ax_size)

### New approach

In [ ]:
x_arr, y_arr = np.meshgrid(xx, yy)
window = normalize(1 - np.sqrt(x_arr ** 2 + y_arr ** 2))
window = window

windowed_img = window * img
window_ft = find_ft_2d(window)
windowed_img_ft = find_ft_2d(windowed_img)
  
magn_windowed = np.abs(windowed_img_ft)
phase_windowed = np.angle(windowed_img_ft)
restored_img = find_ift_2d(magn_windowed * np.exp(1j * phase_windowed)).real

vmin = np.min(windowed_img)
vmax = np.max(windowed_img)

f, ax = show_images(windowed_img, np.log10(magn_windowed), phase_windowed, restored_img, 
                    x_fig_size=x_ax_size, y_fig_size=y_ax_size)

In [ ]:
windowed_img_ft = find_ft_2d(phase_windowed)
m = np.abs(windowed_img_ft)
a = np.angle(windowed_img_ft)

whitenoise = np.random.normal(0, 1, (x_size, y_size))
psd = m * m
restored_phase = find_ift_2d(psd ** 0.5 * np.abs(whitenoise) * np.exp(1j * a)).real

kernel = freq_pink_filter_2d(xx, yy, factor=0.5)
cloud_hf = find_ift_2d(magn_windowed * np.exp(1j * restored_phase)).real
a, b = lin_regression(cloud_hf, img)
cloud_hf = a * cloud_hf + b

f, ax = show_images(np.log10(m), restored_phase, cloud_hf, x_fig_size=x_ax_size, y_fig_size=y_ax_size)

In [ ]:
alpha1 = 1.8
alpha2 = 1.0
alpha_x = 1.4
alpha_y = 1.

whitenoise = np.random.normal(0, 1, (x_size, y_size))
whitenoise_fr = find_ft_2d(whitenoise)
kernel = fit_clement(xx, yy, alpha1, alpha2, eta=0.5)
cloud_lf = find_ift_2d(whitenoise_fr * kernel).real
a, b = lin_regression(cloud_lf, img)
cloud_lf = a * cloud_lf + b

f, ax = show_images(np.log10(kernel), cloud_hf + 0.001 * cloud_lf, x_fig_size=x_ax_size, y_fig_size=y_ax_size)

### Defining an angle between X-axis and ellipse

In [ ]:
weight_exp = 2
threshold = 9
width_x = 5
width_y = 0

# Removing lines along X-axis to get better angle estimation 
min_magn = np.min(magn_windowed)
fit_magn = np.log(magn_windowed / min_magn)

for w in range(-width_y, width_y):
    fit_magn[:, y_shift + w] = fit_magn[:, y_shift + width_y] * 0

for w in range(-width_x, width_x):
    fit_magn[x_shift + w, :] = fit_magn[x_shift + width_x, :]

# Clipping magnitude 
fit_magn = np.where(fit_magn < threshold, 0, fit_magn)
fit_magn_max = np.exp(np.max(fit_magn))
fit_magn = fit_magn / np.mean(fit_magn)

# Defining slope with regression
avg_xy = -np.sum(x * y * fit_magn ** weight_exp)
avg_y2 = np.sum(y * y * fit_magn ** weight_exp)
avg_z2 = np.sum(fit_magn ** weight_exp)
a = avg_xy / avg_y2
angle = np.arctan(a) * 180 / np.pi

# Defining STD along the lines with the found slopes
lperp2 = np.sum((x + a * y) ** 2 * fit_magn ** weight_exp) / avg_z2
lpara2 = np.sum((a * x - y) ** 2 * fit_magn ** weight_exp) / avg_z2
lperp = lperp2 ** 0.5
lpara = lpara2 ** 0.5

# Outputing results 
print(f'A = {a}')
print(f'Angle = {angle}')
print(f'Slope = {1 / a}')
print(f'l_perp = {lperp}')
print(f'l_para = {lpara}')
print(f'l_perp corrected = {lperp2 ** 0.5}')

# Defining lines with found slopes
y_line = np.linspace(50, 850)
x_line = -a * (y_line - y_shift) + x_shift

# Defining gaussian approximation of clipped magnitude
model_magn = np.exp(-(x + a * y) ** 2 / 2 / (lperp2)  -(a * x - y) ** 2 / 2 / (lpara2))
model_magn = model_magn / np.mean(model_magn)

# Plotting 
f, ax = show_images(fit_magn, model_magn)
ax[0].plot(x_line, y_line)
ax[1].plot(x_line, y_line)
plt.show()

### Regression along lines

In [ ]:
# Parallel line
slope = -1 / a
x_line_para = np.linspace(0, img.shape[0], img.shape[0], endpoint=False)
y_line_para = slope * (x_line_para - img.shape[0] // 2) + img.shape[1] // 2

# Clipping arrays
x_mask = (x_line_para < x_size) & (x_line_para > 0)
x_line_para = x_line_para[x_mask]
y_line_para = y_line_para[x_mask]

y_mask = (y_line_para < y_size) & (y_line_para > 0)
x_line_para = x_line_para[y_mask]
y_line_para = y_line_para[y_mask]

# Making arrays to have the same shape as original array
x_line_min, x_line_max = int(np.min(x_line_para)), int(np.max(x_line_para))
x_line_para = np.linspace(x_line_min, x_line_max, img.shape[0], endpoint=False)
y_line_para = slope * (x_line_para - img.shape[0] // 2) + img.shape[1] // 2


# Perpendicular line 
slope = -1 / slope
x_line_perp = np.linspace(0, img.shape[0], img.shape[0], endpoint=False)
y_line_perp = slope * (x_line_perp - img.shape[0] // 2) + img.shape[1] // 2

# Clipping arrays
x_mask = (x_line_perp < x_size) & (x_line_perp > 0)
x_line_perp = x_line_perp[x_mask]
y_line_perp = y_line_perp[x_mask]

y_mask = (y_line_perp < y_size) & (y_line_perp > 0)
x_line_perp = x_line_perp[y_mask]
y_line_perp = y_line_perp[y_mask]

x_line_min, x_line_max = int(np.min(x_line_perp)) + 1, int(np.max(x_line_perp))
x_line_perp = np.linspace(x_line_min, x_line_max, img.shape[0], endpoint=False)
y_line_perp = slope * (x_line_perp - img.shape[0] // 2) + img.shape[1] // 2

f, ax = show_images(np.log10(magn_windowed), cmap='gray', 
                    x_fig_size=x_ax_size/2, y_fig_size=y_ax_size/2)
ax[0].plot(x_line_para, y_line_para)
ax[0].plot(x_line_perp, y_line_perp)

In [ ]:
#1D regression along Y = angle * X line
x_indexes_para = x_line_para.astype(int)[x_shift:]
y_indexes_para = y_line_para.astype(int)[y_shift:]

x_vals_para = np.hypot(x_indexes_para - x_shift, y_indexes_para - y_shift)
y_vals_para = magn_windowed[y_indexes_para, x_indexes_para]
y_vals_para = normalize(y_vals_para)

# Regression variables
x_log_para = np.log(1 + x_vals_para)
y_log_para = np.log(y_vals_para)
alpha_para = -2 * np.sum(y_log_para * x_log_para) / np.sum(x_log_para ** 2) 
alpha_para = alpha_para / 2 

para_approx = (1 + x_vals_para) ** (-alpha_para)
para_approx /= np.max(para_approx)

plt.plot(x_vals_para, y_vals_para)
plt.plot(x_vals_para + 1, para_approx)
plt.yscale('log')

print(alpha_para)

### Playground

In [ ]:
# Stretching gaussain along axes
# Define the size of the kernel and standard deviation
kernel_size = 21
sigma = 2.0

# Create a standard 2D Gaussian kernel
x = np.linspace(-kernel_size // 2, kernel_size // 2, kernel_size)
y = np.linspace(-kernel_size // 2, kernel_size // 2, kernel_size)
x, y = np.meshgrid(x, y)
kernel = np.exp(-(x**2 + y**2) / (2.0 * sigma**2))

# Create a scaling factor for stretching along y = x
stretching_factor = 2.0  # Adjust this value to control the stretching
new_x = stretching_factor * x

# Create the stretched Gaussian kernel
stretched_kernel = np.exp(-(new_x**2 + y**2) / (2.0 * sigma**2))

# Display the original and stretched kernels
plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.imshow(kernel, cmap='viridis')
plt.title('Original Gaussian Kernel')

plt.subplot(1, 2, 2)
plt.imshow(stretched_kernel, cmap='viridis')
plt.title('Stretched Gaussian Kernel')
plt.show()

In [ ]:
# Some experiments
# Blending whitenoise phase with original phase

whitenoise = np.random.normal(0, 1, (x_size, y_size))
whitenoise_fr = find_ft_2d(whitenoise)
new_angle = np.angle(whitenoise_fr)

width = 25 #img.shape[0] // 2 - 1
step = 1

for i in range(-width, width + 1, step):
    new_angle[img.shape[0] // 2 + i, :] = phase_windowed[img.shape[0] // 2 + i, :]
    
for j in range(-width, width + 1, step):
    new_angle[:, img.shape[1] // 2 + j] = phase_windowed[:, img.shape[1] // 2 + j]
        
        
dunno = find_ift_2d(magn_windowed * np.exp(1j * new_angle)).real 
# f, ax = show_images(dunno / window)

# sharp_filter = 1 - (freq_sharp_round_filter_2d(xx, yy, radius=900) - freq_sharp_round_filter_2d(xx, yy, radius=50))
# dunno = find_ift_2d(sharp_filter * magn_windowed * np.exp(1j * phase_windowed)).real
dunno = find_ift_2d(magn_windowed * np.exp(1j * new_angle)).real
a, b = lin_regression(dunno, img)
dunno = a * dunno + b

f, ax = show_images(new_angle, dunno)